In [1]:
# %%
import os
import sys

# Add the path to the directory containing UNET.py (only if it is in a different directory)
sys.path.append('/content/drive/MyDrive/')

from tensorflow.python.keras.utils.version_utils import callbacks

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from UNET import build_unet # Or try: from .UNET import build_unet
from metrics import dice_loss, dice_coef

In [2]:
""" Global parameters """

H = 256
W = 256

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)


In [3]:
def load_dataset(path, split=0.2):
    images = sorted(glob(os.path.join(path, "images", "*.png")))
    masks = sorted(glob(os.path.join(path, "masks", "*.png")))
    #print(images[0], masks[0])

    split_size = int(len(images) * split)

    train_x, valid_x = train_test_split(images, test_size=split_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=split_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=split_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=split_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)


In [4]:
def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    x= x/255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  #(h,w)
    x = cv2.resize(x, (W, H))  #(h,w)
    x= x/255.0  #(h,w)
    x= x.astype(np.float32)  #(h,w)
    x = np.expand_dims(x, axis = -1)  #(h,w,1)

    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])  # <-- Assign output

    x.set_shape((H, W, 3))  # <-- Explicitly set the shape after assignment
    y.set_shape((H, W, 1))

    return x, y


In [5]:
def tf_dataset(X, Y, batch = 2):
    dataset = tf.data.Dataset.from_tensor_slices((X,Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)

    return dataset

if __name__ == "__main__":
    """Seeding"""
    np.random.seed(42)
    tf.random.set_seed(42)

    """directory for storing files"""
    create_dir("files")

    """Hyperparameters"""
    batch_size = 16
    lr = 1e-4
    num_epochs = 500
    model_path = os.path.join("files", "model.h5")
    csv_path = os.path.join("files", "log.csv")

    """Dataset"""
    dataset_path = "/content/drive/MyDrive/SegmentationDataset"
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_dataset(dataset_path)

    print(f"Train: {len(train_x)} - {len(train_y)}")
    print(f"Valid: {len(valid_x)} - {len(valid_y)}")
    print(f"Test: {len(test_x)} - {len(test_y)}")


    train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y, batch = batch_size)

    #for x, y in train_dataset:
    #    print(x.shape, y.shape)

    """Model"""
    model = build_unet((H, W, 3))
    model.compile(loss=dice_loss, optimizer = Adam(lr), metrics = [dice_coef])

    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path),
        EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False),
    ]

    model.fit(
        train_dataset,
        epochs=num_epochs,
        validation_data=valid_dataset,
        callbacks=callbacks
    )


Train: 1840 - 1840
Valid: 612 - 612
Test: 612 - 612
Epoch 1/500
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - dice_coef: 0.0788 - loss: 0.9212 
Epoch 1: val_loss improved from inf to 0.95465, saving model to files/model.h5


115/115 ━━━━━━━━━━━━━━━━━━━━ 2088s 17s/step - dice_coef: 0.0791 - loss: 0.9209 - val_dice_coef: 0.0450 - val_loss: 0.9547 - learning_rate: 1.0000e-04
Epoch 2/500
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 844ms/step - dice_coef: 0.1854 - loss: 0.8146
Epoch 2: val_loss did not improve from 0.95465
115/115 ━━━━━━━━━━━━━━━━━━━━ 156s 950ms/step - dice_coef: 0.1856 - loss: 0.8144 - val_dice_coef: 0.0240 - val_loss: 0.9759 - learning_rate: 1.0000e-04
Epoch 3/500
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 843ms/step - dice_coef: 0.2447 - loss: 0.7553
Epoch 3: val_loss improved from 0.95465 to 0.94245, saving model to files/model.h5


115/115 ━━━━━━━━━━━━━━━━━━━━ 151s 1s/step - dice_coef: 0.2448 - loss: 0.7552 - val_dice_coef: 0.0571 - val_loss: 0.9424 - learning_rate: 1.0000e-04
Epoch 4/500
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 845ms/step - dice_coef: 0.3107 - loss: 0.6893
Epoch 4: val_loss improved from 0.94245 to 0.89660, saving model to files/model.h5


115/115 ━━━━━━━━━━━━━━━━━━━━ 136s 979ms/step - dice_coef: 0.3108 - loss: 0.6892 - val_dice_coef: 0.1021 - val_loss: 0.8966 - learning_rate: 1.0000e-04
Epoch 5/500
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 840ms/step - dice_coef: 0.3844 - loss: 0.6156
Epoch 5: val_loss improved from 0.89660 to 0.70645, saving model to files/model.h5


115/115 ━━━━━━━━━━━━━━━━━━━━ 121s 1s/step - dice_coef: 0.3845 - loss: 0.6155 - val_dice_coef: 0.2937 - val_loss: 0.7064 - learning_rate: 1.0000e-04
Epoch 6/500
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 844ms/step - dice_coef: 0.4588 - loss: 0.5412
Epoch 6: val_loss improved from 0.70645 to 0.55169, saving model to files/model.h5


...

115/115 ━━━━━━━━━━━━━━━━━━━━ 142s 964ms/step - dice_coef: 0.8065 - loss: 0.1935 - val_dice_coef: 0.7329 - val_loss: 0.2640 - learning_rate: 1.0000e-04
Epoch 16/500
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 845ms/step - dice_coef: 0.8311 - loss: 0.1689
Epoch 16: val_loss did not improve from 0.26403
115/115 ━━━━━━━━━━━━━━━━━━━━ 141s 953ms/step - dice_coef: 0.8310 - loss: 0.1690 - val_dice_coef: 0.6640 - val_loss: 0.3296 - learning_rate: 1.0000e-04
Epoch 17/500
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 845ms/step - dice_coef: 0.8374 - loss: 0.1626
Epoch 17: val_loss did not improve from 0.26403
115/115 ━━━━━━━━━━━━━━━━━━━━ 142s 953ms/step - dice_coef: 0.8373 - loss: 0.1627 - val_dice_coef: 0.7031 - val_loss: 0.2966 - learning_rate: 1.0000e-04
Epoch 18/500
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 842ms/step - dice_coef: 0.8437 - loss: 0.1563
Epoch 18: val_loss improved from 0.26403 to 0.26237, saving model to files/model.h5


115/115 ━━━━━━━━━━━━━━━━━━━━ 119s 1s/step - dice_coef: 0.8437 - loss: 0.1563 - val_dice_coef: 0.7363 - val_loss: 0.2624 - learning_rate: 1.0000e-04
Epoch 19/500
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 841ms/step - dice_coef: 0.8489 - loss: 0.1511
Epoch 19: val_loss improved from 0.26237 to 0.23588, saving model to files/model.h5


115/115 ━━━━━━━━━━━━━━━━━━━━ 113s 980ms/step - dice_coef: 0.8490 - loss: 0.1510 - val_dice_coef: 0.7624 - val_loss: 0.2359 - learning_rate: 1.0000e-04
Epoch 20/500
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 845ms/step - dice_coef: 0.8708 - loss: 0.1292
Epoch 20: val_loss improved from 0.23588 to 0.22011, saving model to files/model.h5


115/115 ━━━━━━━━━━━━━━━━━━━━ 140s 967ms/step - dice_coef: 0.8708 - loss: 0.1292 - val_dice_coef: 0.7787 - val_loss: 0.2201 - learning_rate: 1.0000e-04
Epoch 21/500
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 841ms/step - dice_coef: 0.8759 - loss: 0.1241
Epoch 21: val_loss did not improve from 0.22011
115/115 ━━━━━━━━━━━━━━━━━━━━ 110s 948ms/step - dice_coef: 0.8759 - loss: 0.1241 - val_dice_coef: 0.7714 - val_loss: 0.2245 - learning_rate: 1.0000e-04
Epoch 22/500
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 846ms/step - dice_coef: 0.8809 - loss: 0.1191
Epoch 22: val_loss did not improve from 0.22011
115/115 ━━━━━━━━━━━━━━━━━━━━ 150s 1s/step - dice_coef: 0.8809 - loss: 0.1191 - val_dice_coef: 0.7778 - val_loss: 0.2225 - learning_rate: 1.0000e-04
Epoch 23/500
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 846ms/step - dice_coef: 0.8878 - loss: 0.1122
Epoch 23: val_loss did not improve from 0.22011
115/115 ━━━━━━━━━━━━━━━━━━━━ 134s 955ms/step - dice_coef: 0.8878 - loss: 0.1122 - val_dice_coef: 0.7709 - val_loss: 0.2302 - learning

115/115 ━━━━━━━━━━━━━━━━━━━━ 152s 1s/step - dice_coef: 0.8951 - loss: 0.1049 - val_dice_coef: 0.8097 - val_loss: 0.1885 - learning_rate: 1.0000e-05
Epoch 27/500
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 842ms/step - dice_coef: 0.9078 - loss: 0.0922
Epoch 27: val_loss did not improve from 0.18854
115/115 ━━━━━━━━━━━━━━━━━━━━ 109s 948ms/step - dice_coef: 0.9078 - loss: 0.0922 - val_dice_coef: 0.8076 - val_loss: 0.1908 - learning_rate: 1.0000e-05
Epoch 28/500
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 840ms/step - dice_coef: 0.9117 - loss: 0.0883
Epoch 28: val_loss did not improve from 0.18854
115/115 ━━━━━━━━━━━━━━━━━━━━ 109s 946ms/step - dice_coef: 0.9117 - loss: 0.0883 - val_dice_coef: 0.8079 - val_loss: 0.1908 - learning_rate: 1.0000e-05
Epoch 29/500
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 845ms/step - dice_coef: 0.9140 - loss: 0.0860
Epoch 29: val_loss did not improve from 0.18854
115/115 ━━━━━━━━━━━━━━━━━━━━ 143s 952ms/step - dice_coef: 0.9140 - loss: 0.0860 - val_dice_coef: 0.8081 - val_loss: 0.1907 - learning